In [2]:
# Examples of Pyspark ML
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

22/04/05 11:15:42 WARN Utils: Your hostname, DESKTOP-HGFEG13 resolves to a loopback address: 127.0.1.1; using 172.22.7.66 instead (on interface eth0)
22/04/05 11:15:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/05 11:15:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
### read the dataset
training = spark.read.csv('../data/demo_data_5.csv', header=True, inferSchema = True)

In [10]:
training.show()

+--------+---+----------+------+
|    name|age|experience|salary|
+--------+---+----------+------+
|    Guts| 31|        10| 30000|
|   Casca| 30|         8| 25000|
|  Judeau| 29|         4| 20000|
|Griffith| 24|         3| 20000|
|  Pippin| 21|         1| 15000|
|  Corkus| 23|         2| 18000|
+--------+---+----------+------+



In [11]:
training.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [12]:
training.columns

['name', 'age', 'experience', 'salary']

In [ ]:
#[age, experience] ----> new feature ----> independent feature

In [14]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols=['age', 'experience'], outputCol = 'Independent Features')

In [15]:
output = feature_assembler.transform(training)

In [16]:
output.show()

+--------+---+----------+------+--------------------+
|    name|age|experience|salary|Independent Features|
+--------+---+----------+------+--------------------+
|    Guts| 31|        10| 30000|         [31.0,10.0]|
|   Casca| 30|         8| 25000|          [30.0,8.0]|
|  Judeau| 29|         4| 20000|          [29.0,4.0]|
|Griffith| 24|         3| 20000|          [24.0,3.0]|
|  Pippin| 21|         1| 15000|          [21.0,1.0]|
|  Corkus| 23|         2| 18000|          [23.0,2.0]|
+--------+---+----------+------+--------------------+



In [17]:
output.columns

['name', 'age', 'experience', 'salary', 'Independent Features']

In [18]:
finalized_data = output.select('Independent Features','Salary')

In [19]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [26]:
from pyspark.ml.regression import LinearRegression

## Train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = 'Independent Features', labelCol = 'Salary')
regressor = regressor.fit(train_data)

22/04/05 11:29:45 WARN Instrumentation: [169845cf] regParam is zero, which might cause numerical instability and overfitting.


In [27]:
regressor.coefficients

DenseVector([-0.0, 1666.6667])

In [29]:
# Intercepts
regressor.intercept

13333.333333333807

In [32]:
# Predictions
pred_results = regressor.evaluate(test_data)

In [33]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|16666.666666666693|
|          [24.0,3.0]| 20000| 18333.33333333336|
|          [30.0,8.0]| 25000|26666.666666666664|
+--------------------+------+------------------+



/home/thiago/miniconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [34]:
pred_results.meanAbsoluteError

1555.5555555555366